<a href="https://colab.research.google.com/github/asyaf/fun_mini_projects/blob/master/image_route_viz/Route_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/asyaf/fun_mini_projects

Cloning into 'fun_mini_projects'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 100 (delta 25), reused 25 (delta 1), pack-reused 0
Receiving objects: 100% (100/100), 46.78 MiB | 12.22 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:
import base64
import folium
import glob
import io
import os

from IPython.display import display
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [ ]:
DATA_DIR = 'fun_mini_projects/image_route_viz/example'
IMG_EXT = '.jpg'

GPS_EXIF_TAG = 'GPSInfo'
GPS_LAT_TAG = 'GPSLatitude'
GPS_LAT_REF_TAG = 'GPSLatitudeRef'
GPS_LONG_TAG = 'GPSLongitude'
GPS_LONG_REF_TAG = 'GPSLongitudeRef'

JERUSALEM_COORD = [31.7683, 35.2137]
DEFAULT_ZOOM = 7
POPUP_IMAGE_SIZE = (200, 100) 
POPUP_MARGIN = 50

In [ ]:
def dms_to_dec(dms, ref):
    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def gps_info_to_coord(gps_info):
    lat = dms_to_dec(gps_info[GPS_LAT_TAG], gps_info[GPS_LAT_REF_TAG])
    lon = dms_to_dec(gps_info[GPS_LONG_TAG], gps_info[GPS_LONG_REF_TAG])
    return (lat,lon)

def get_gps_img_data(exif_data):
    for tag_id in exif_data:
        # get the tag name, instead of human unreadable tag id
        tag = TAGS.get(tag_id)
        if tag == GPS_EXIF_TAG:
            data = exif_data.get(tag_id)
            return data
    return None

def gps_info_to_dict(gps_data):
    gps_dict = {}
    for tag_id, value in GPSTAGS.items():
        if tag_id in gps_data:
            gps_dict[value] = gps_data[tag_id]
    return gps_dict

In [ ]:
def get_images_in_dir(dirname):
    glob_expr = os.path.join(dirname, '*{}'.format(IMG_EXT))
    img_files = glob.glob(glob_expr)
    return img_files
    
def load_images(img_files):
    images = []
    for img_file in img_files:
        img = Image.open(img_file)
        images.append(img)
    return images

def get_images_coords(images):
    coords = []
    for img in images:
        exif_data = img.getexif()
        gps_data = get_gps_img_data(exif_data)
        gps_dict = gps_info_to_dict(gps_data)
        if gps_data is not None and len(gps_data) > 0:
            lat, lon = gps_info_to_coord(gps_dict)
            coords.append((lat, lon))
        else:
            # no gps data found
            coords.append((None, None))
    return coords

In [ ]:
image_files = get_images_in_dir(DATA_DIR)
images = load_images(image_files)

In [ ]:
coords = get_images_coords(images)

In [ ]:
def add_image_marker(ind, img, coord):
    resized = img.resize(POPUP_IMAGE_SIZE)
    html = '<figure>'
    img_byte_arr = io.BytesIO()
    resized.save(img_byte_arr, format=img.format)
    encoded = base64.b64encode(img_byte_arr.getvalue())
    html += '<img src="data:image/jpeg;base64,{}">'.format(encoded.decode('UTF-8'))
    html += '<figcaption>{}</figcaption></figure>'.format(ind)
    width = POPUP_IMAGE_SIZE[0] + POPUP_MARGIN
    height = POPUP_IMAGE_SIZE[1] + POPUP_MARGIN
    iframe = folium.IFrame(html, width=width, height=height)
    tooltip = folium.Popup(iframe)
    return folium.Marker(coord,popup=tooltip)

In [ ]:
fmap = folium.Map(location=JERUSALEM_COORD, zoom_start=DEFAULT_ZOOM)
for i, (img, coord) in enumerate(zip(images, coords)):
    if coord[0] is not None:
        marker = add_image_marker(i, img, coord)
        marker.add_to(fmap)
fmap